In [1]:
'''
Overview
this notebook parses all past ufc fight stats when run, it does not include upcoming fights


scrape ufc fight stats
get all event details, name, url, date, location for all ufc events
for each event, get fight details all fights on card
parse each fight to get fight stats of both fighters
'''

'\nOverview\nthis notebook parses all past ufc fight stats when run, it does not include upcoming fights\n\n\nscrape ufc fight stats\nget all event details, name, url, date, location for all ufc events\nfor each event, get fight details all fights on card\nparse each fight to get fight stats of both fighters\n'

In [18]:
# imports
import pandas as pd
from tqdm.notebook import tqdm_notebook
import time

# import library
import scrape_ufc_stats_library as LIB

# import config
import yaml
config = yaml.safe_load(open('scrape_ufc_stats_config.yaml'))

# Parse Event Details
Includes:
<br>
Event
<br>
URL
<br>
Date
<br>
Location
<br>

In [3]:
# define url to parse
events_url = config['completed_events_all_url']

In [4]:
# get soup
soup = LIB.get_soup(events_url)

# parse event details
all_event_details_df = LIB.parse_event_details(soup)

# show event details
display(all_event_details_df)

# write event details to file
all_event_details_df.to_csv(config['event_details_file_name'], index=False)

,EVENT,URL,DATE,LOCATION
0,UFC Fight Night: Holloway vs. Allen,http://ufcstats.com/event-details/15b1b21cd743...,"April 15, 2023","Kansas City, Missouri, USA"
1,UFC 287: Pereira vs. Adesanya 2,http://ufcstats.com/event-details/3dc3022232b7...,"April 08, 2023","Miami, Florida, USA"
2,UFC Fight Night: Vera vs. Sandhagen,http://ufcstats.com/event-details/aec273fcb765...,"March 25, 2023","San Antonio, Texas, USA"
3,UFC 286: Edwards vs. Usman 3,http://ufcstats.com/event-details/e4bb7e483c4a...,"March 18, 2023","London, England, United Kingdom"
4,UFC Fight Night: Yan vs. Dvalishvili,http://ufcstats.com/event-details/35080a7f406f...,"March 11, 2023","Las Vegas, Nevada, USA"
...,...,...,...,...
637,UFC 6: Clash of the Titans,http://ufcstats.com/event-details/1c3f5e85b59e...,"July 14, 1995","Casper, Wyoming, USA"
638,UFC 5: The Return of the Beast,http://ufcstats.com/event-details/dedc3bb440d0...,"April 07, 1995","Charlotte, North Carolina, USA"
639,UFC 4: Revenge of the Warriors,http://ufcstats.com/event-details/b60391da771d...,"December 16, 1994","Tulsa, Oklahoma, USA"
640,UFC 3: The American Dream,http://ufcstats.com/event-details/1a49e0670dfa...,"September 09, 1994","Charlotte, North Carolina, USA"


# Parse Fight Details
Includes:
<br>
Event
<br>
Bout
<br>
URL

In [5]:
# define list of urls of fights to parse
list_of_events_urls = list(all_event_details_df['URL'])

In [6]:
# create empty df to store fight details
all_fight_details_df = pd.DataFrame(columns=config['fight_details_column_names'])

# loop through each event and parse fight details
for url in tqdm_notebook(list_of_events_urls):

    # get soup
    soup = LIB.get_soup(url)

    # parse fight links
    fight_details_df = LIB.parse_fight_details(soup)
    fight_details_df['eventURL'] = url
    
    # concat fight details
    all_fight_details_df = pd.concat([all_fight_details_df, fight_details_df])

# show all fight details
display(all_fight_details_df)

# write fight details to file
all_fight_details_df.to_csv(config['fight_details_file_name'], index=False)

  0%|          | 0/642 [00:00<?, ?it/s]

,EVENT,BOUT,URL,eventURL
0,UFC Fight Night: Holloway vs. Allen,Max Holloway vs. Arnold Allen,http://ufcstats.com/fight-details/6049301b4dee...,http://ufcstats.com/event-details/15b1b21cd743...
1,UFC Fight Night: Holloway vs. Allen,Edson Barboza vs. Billy Quarantillo,http://ufcstats.com/fight-details/83c81018bb3a...,http://ufcstats.com/event-details/15b1b21cd743...
2,UFC Fight Night: Holloway vs. Allen,Azamat Murzakanov vs. Dustin Jacoby,http://ufcstats.com/fight-details/792f139c2b9f...,http://ufcstats.com/event-details/15b1b21cd743...
3,UFC Fight Night: Holloway vs. Allen,Ion Cutelaba vs. Tanner Boser,http://ufcstats.com/fight-details/995e79b951da...,http://ufcstats.com/event-details/15b1b21cd743...
4,UFC Fight Night: Holloway vs. Allen,Pedro Munhoz vs. Chris Gutierrez,http://ufcstats.com/fight-details/d856812dc733...,http://ufcstats.com/event-details/15b1b21cd743...
...,...,...,...,...
10,UFC 2: No Way Out,Orlando Wiet vs. Robert Lucarelli,http://ufcstats.com/fight-details/3b020d4914b4...,http://ufcstats.com/event-details/a6a9ab5a824e...
11,UFC 2: No Way Out,Frank Hamaker vs. Thaddeus Luster,http://ufcstats.com/fight-details/d917c8c7461b...,http://ufcstats.com/event-details/a6a9ab5a824e...
12,UFC 2: No Way Out,Johnny Rhodes vs. David Levicki,http://ufcstats.com/fight-details/ccee020be2e8...,http://ufcstats.com/event-details/a6a9ab5a824e...
13,UFC 2: No Way Out,Patrick Smith vs. Ray Wizard,http://ufcstats.com/fight-details/4b9ae533ccb3...,http://ufcstats.com/event-details/a6a9ab5a824e...


# Parse Fight Results and Fight Stats

Fight Results Includes:
<br>
Event
<br>
Bout
<br>
Weightclass
<br>
Method
<br>
Round
<br>
Time
<br>
Time Format
<br>
Referee
<br>
Details
<br>

Fight Stats Includes:
<br>
Event
<br>
Bout
<br>
Round
<br>
Fighter
<br>
Kd
<br>
Sig.Str.
<br>
Sig.Str. %
<br>
Total Str.
<br>
Td
<br>
Td %
<br>
Sub.Att
<br>
Rev.
<br>
Ctrl
<br>
Head
<br>
Body
<br>
Leg
<br>
Distance
<br>
Clinch
<br>
Ground
<br>

In [14]:
# define list of urls of fights to parse
list_of_fight_details_urls = list(all_fight_details_df['URL'])

In [24]:
# create empty df to store fight results
all_fight_results_df = pd.DataFrame(columns=config['fight_results_column_names'])
# create empty df to store fight stats
all_fight_stats_df = pd.DataFrame(columns=config['fight_stats_column_names'])

# loop through each fight and parse fight results and stats
for url in tqdm_notebook(list_of_fight_details_urls):
    backoff = 1
    while True:
        try:
            # get soup
            soup = LIB.get_soup(url)

            # parse fight results and fight stats
            fight_results_df, fight_stats_df = LIB.parse_organise_fight_results_and_stats(
                soup,
                url,
                config['fight_results_column_names'],
                config['totals_column_names'],
                config['significant_strikes_column_names']
            )

            # concat fight results
            all_fight_results_df = pd.concat([all_fight_results_df, fight_results_df])
            # concat fight stats
            all_fight_stats_df = pd.concat([all_fight_stats_df, fight_stats_df])
            break
        except Exception as e:
            print(f'threw {e}. Sleeping for {backoff} seconds and retrying...')
            time.sleep(backoff)
            backoff = min(16, 2 * backoff)

# show all fight results
display(all_fight_results_df)
# show all fight stats
display(all_fight_stats_df)

# write to file
all_fight_results_df.to_csv(config['fight_results_file_name'], index=False)
# write to file
all_fight_stats_df.to_csv(config['fight_stats_file_name'], index=False)

  0%|          | 0/7086 [00:00<?, ?it/s]

,EVENT,BOUT,OUTCOME,WEIGHTCLASS,METHOD,ROUND,TIME,TIME FORMAT,REFEREE,DETAILS,URL
0,UFC Fight Night: Holloway vs. Allen,Max Holloway vs. Arnold Allen,W/L,Featherweight Bout,Decision - Unanimous,5,5:00,5 Rnd (5-5-5-5-5),Jason Herzog,Chris Lee 46 - 49.Sal D'amato 46 - 49.Travis B...,http://ufcstats.com/fight-details/6049301b4dee...
0,UFC Fight Night: Holloway vs. Allen,Edson Barboza vs. Billy Quarantillo,W/L,Featherweight Bout,KO/TKO,1,2:37,3 Rnd (5-5-5),Keith Peterson,Knee to Head At Distance,http://ufcstats.com/fight-details/83c81018bb3a...
0,UFC Fight Night: Holloway vs. Allen,Dustin Jacoby vs. Azamat Murzakanov,L/W,Light Heavyweight Bout,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Nick Berens,Sal D'amato 28 - 29.Brian Puccillo 28 - 29.Ros...,http://ufcstats.com/fight-details/792f139c2b9f...
0,UFC Fight Night: Holloway vs. Allen,Tanner Boser vs. Ion Cutelaba,L/W,Light Heavyweight Bout,KO/TKO,1,2:05,3 Rnd (5-5-5),Keith Peterson,Punches to Head On Ground,http://ufcstats.com/fight-details/995e79b951da...
0,UFC Fight Night: Holloway vs. Allen,Pedro Munhoz vs. Chris Gutierrez,W/L,Bantamweight Bout,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Jason Herzog,Brian Puccillo 27 - 30.Travis Buesking 27 - 30...,http://ufcstats.com/fight-details/d856812dc733...
...,...,...,...,...,...,...,...,...,...,...,...
0,UFC 2: No Way Out,Orlando Wiet vs. Robert Lucarelli,W/L,Open Weight Bout,KO/TKO,1,2:50,No Time Limit,John McCarthy,toCorner Stoppage,http://ufcstats.com/fight-details/3b020d4914b4...
0,UFC 2: No Way Out,Frank Hamaker vs. Thaddeus Luster,W/L,Open Weight Bout,Submission,1,4:52,No Time Limit,John McCarthy,Keylock From Half Guard,http://ufcstats.com/fight-details/d917c8c7461b...
0,UFC 2: No Way Out,Johnny Rhodes vs. David Levicki,W/L,Open Weight Bout,KO/TKO,1,12:13,No Time Limit,John McCarthy,Punches to Head From GuardSubmission to Strikes,http://ufcstats.com/fight-details/ccee020be2e8...
0,UFC 2: No Way Out,Patrick Smith vs. Ray Wizard,W/L,Open Weight Bout,Submission,1,0:58,No Time Limit,John McCarthy,Guillotine Choke Standing,http://ufcstats.com/fight-details/4b9ae533ccb3...


,EVENT,BOUT,ROUND,FIGHTER,KD,SIG.STR.,SIG.STR. %,TOTAL STR.,TD,TD %,...,REV.,CTRL,HEAD,BODY,LEG,DISTANCE,CLINCH,GROUND,fighterURL,fightURL
0,UFC Fight Night: Holloway vs. Allen,Max Holloway vs. Arnold Allen,Round 1,Max Holloway,0,27 of 62,43%,27 of 62,0 of 0,---,...,0,0:00,13 of 46,9 of 11,5 of 5,27 of 62,0 of 0,0 of 0,http://ufcstats.com/fighter-details/150ff4cc64...,http://ufcstats.com/fight-details/6049301b4dee...
1,UFC Fight Night: Holloway vs. Allen,Max Holloway vs. Arnold Allen,Round 2,Max Holloway,0,25 of 49,51%,25 of 49,0 of 0,---,...,0,0:00,11 of 30,8 of 9,6 of 10,25 of 49,0 of 0,0 of 0,http://ufcstats.com/fighter-details/150ff4cc64...,http://ufcstats.com/fight-details/6049301b4dee...
2,UFC Fight Night: Holloway vs. Allen,Max Holloway vs. Arnold Allen,Round 3,Max Holloway,0,33 of 51,64%,33 of 51,0 of 0,---,...,0,0:00,20 of 37,7 of 7,6 of 7,33 of 51,0 of 0,0 of 0,http://ufcstats.com/fighter-details/150ff4cc64...,http://ufcstats.com/fight-details/6049301b4dee...
3,UFC Fight Night: Holloway vs. Allen,Max Holloway vs. Arnold Allen,Round 4,Max Holloway,0,29 of 52,55%,29 of 52,0 of 0,---,...,0,0:00,12 of 33,10 of 12,7 of 7,29 of 52,0 of 0,0 of 0,http://ufcstats.com/fighter-details/150ff4cc64...,http://ufcstats.com/fight-details/6049301b4dee...
4,UFC Fight Night: Holloway vs. Allen,Max Holloway vs. Arnold Allen,Round 5,Max Holloway,0,33 of 48,68%,35 of 50,0 of 0,---,...,0,0:03,21 of 34,8 of 9,4 of 5,32 of 47,1 of 1,0 of 0,http://ufcstats.com/fighter-details/150ff4cc64...,http://ufcstats.com/fight-details/6049301b4dee...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,UFC 2: No Way Out,Johnny Rhodes vs. David Levicki,Round 1,David Levicki,0,4 of 5,80%,95 of 102,0 of 0,---,...,0,--,4 of 5,0 of 0,0 of 0,1 of 2,2 of 2,1 of 1,http://ufcstats.com/fighter-details/49590e0508...,http://ufcstats.com/fight-details/ccee020be2e8...
0,UFC 2: No Way Out,Patrick Smith vs. Ray Wizard,Round 1,Patrick Smith,0,1 of 1,100%,1 of 1,0 of 1,0%,...,0,--,0 of 0,1 of 1,0 of 0,0 of 0,1 of 1,0 of 0,http://ufcstats.com/fighter-details/46c8ec317a...,http://ufcstats.com/fight-details/4b9ae533ccb3...
0,UFC 2: No Way Out,Patrick Smith vs. Ray Wizard,Round 1,Ray Wizard,0,1 of 1,100%,2 of 2,0 of 0,---,...,0,--,0 of 0,0 of 0,1 of 1,1 of 1,0 of 0,0 of 0,http://ufcstats.com/fighter-details/ea0ad15545...,http://ufcstats.com/fight-details/4b9ae533ccb3...
0,UFC 2: No Way Out,Scott Morris vs. Sean Daugherty,Round 1,Scott Morris,0,1 of 1,100%,2 of 2,1 of 1,100%,...,0,--,1 of 1,0 of 0,0 of 0,0 of 0,1 of 1,0 of 0,http://ufcstats.com/fighter-details/be9d259be0...,http://ufcstats.com/fight-details/4acab67848e7...
